# APS360 Group Project

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import SubsetRandomSampler


## Dataset Loading

In [41]:
DATASET_PATH = "./fonts_image_dataset"

def load_data(batch_size):

    # Convert the images to tensors and normalize them
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), transforms.Grayscale(num_output_channels=1)])
    gestures_dataset = torchvision.datasets.ImageFolder(root = DATASET_PATH, transform=transform)

    # Create a list of indices for all the images in the dataset
    dataset_size = len(gestures_dataset)
    indices = list(range(dataset_size))

    np.random.seed(0)
    np.random.shuffle(indices)

    # Split the indices into 60% Training 20% Validation 20% Testing. We need most of the data for training the network, but we must also set aside a bit for validation to fine tune the network, and test the network at the very end.
    split1 = int(0.6 * dataset_size)
    split2 = int(0.8 * dataset_size)
    train_indices, val_indices, test_indices = indices[:split1], indices[split1:split2], indices[split2:]

    # Create a sampler for the training, validation, and testing sets
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)
    test_sampler = SubsetRandomSampler(test_indices)

    # Create the dataloaders for the training, validation, and testing sets
    train_loader = torch.utils.data.DataLoader(gestures_dataset, batch_size=batch_size,sampler=train_sampler)
    val_loader = torch.utils.data.DataLoader(gestures_dataset, batch_size=batch_size,sampler=val_sampler)
    test_loader = torch.utils.data.DataLoader(gestures_dataset, batch_size=batch_size,sampler=test_sampler)

    print("Done Loading Data")

    return train_loader, val_loader, test_loader, gestures_dataset.classes

## Baseline Model

In [42]:
# Extracts alot of features since fonts are written in different letters and font patterns may be more subtle

class BaselineModel(nn.Module):
    
    def __init__(self):
        super(BaselineModel, self).__init__()
        #can take out conv layer 5 &/4 and the size of x.view will remain the same (k*27*27)
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.bn1 = nn.BatchNorm2d(6)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 64, 3, padding = 1) #padding added so that map size is not too small at end of convolution
        self.bn3 = nn.BatchNorm2d(64)
        # self.conv4 = nn.conv2d(64, 128, 3, padding = 1)
        # self.bn4 = nn.BatchNorm2d(128)
        # self.conv5 = nn.conv2d(128, 192, 1, padding = 1)
        # self.bn5 = nn.BatchNorm2d(192)
        self.fc1 = nn.Linear(64*27*27, 15)
        self.fc2 = nn.Linear(15, 30)
        self.fc3 = nn.Linear(30, 10)
        self.dropout = nn.Dropout(0.25) #set training = True in forward to train and use dropout

    def forward(self, x, training = False):
        #convolutional layers
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        # x = F.relu(self.bn4(self.conv4(x)))
        # x = F.relu(self.bn5(self.conv5(x)))

        x = x.view(-1, 64*27*27)  # Flatten for fc layers

        #fully connected layers
        x = F.relu(self.fc1(x))
        if training:
            x = self.dropout(x)
        x = F.relu(self.fc2(x))
        if training:
            x = self.dropout(x)
        x = self.fc3(x)
        return x

    print('Baseline Model Architecture done')


Baseline Model Architecture done


Training & Output Visualization

In [43]:
#Eshan, you can play with these functions and the num of layers in the baseline model for training speed maintaining relatively acceptable accuracy if you want to

def get_accuracy_baseline(model, data_loader):
  correct = 0
  total = 0

  with torch.no_grad():
    for images, labels in data_loader:
      outputs = model(images)
      prediction = outputs.max(1, keepdim=True)[1] #get the index of the max log_probability
      total += labels.size(0)                      #add how many images were predicted
      correct += prediction.eq(labels.view_as(prediction)).sum().item() #add how many were correct

  return correct / total

def train_baseline(model, lr, num_epochs, train_loader, val_loader):
  #initializing lists for plotting later
  iterations = []
  epochs = []
  losses = []
  train_accuracy = []
  val_accuracy = []
  max_val_acc = -1 #to save the best model later

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=lr)

  e = 0 #epoch counter
  n = 0 #iteration counter
  for epoch in range(num_epochs):
    for imgs, labels in iter(train_loader):
      model.train() #set model to training mode. turn on batch norm to try to prevent overfitting
      optimizer.zero_grad() #reset gradients to zero (does not affect internally calculated momentums of optimizer)
      outputs = model(imgs, training = True) #forward pass
      loss = criterion(outputs, labels) #compute loss
      loss.backward() #backward pass
      optimizer.step() #update weights

      #for later plotting of training visualization & to see if overfitting occurs / sanity checks
      model.eval() #set model to evaluation mode. turn off batch norm
      iterations.append(n)
      losses.append(loss.item()) #get loss value
      train_accuracy.append(get_accuracy_baseline(model, train_loader)) #get accuracy on training set
      val_acc = get_accuracy_baseline(model, val_loader) #get accuracy on validation set (a number to be saved)
      val_accuracy.append(val_acc) #get accuracy on validation set
      #for sake of training time, if needed can only calculate accuracies after each epoch; move accuracy calculation and append to outer loop before epochs.append(e) & change plotting section (anything that says iterations can become epochs including graph titles)
      n += 1 #increment iteration counter

      #saving the best model
      if val_acc > max_val_acc:
        max_val_acc = val_acc
        torch.save(model.state_dict(), 'best_baseline_model') #save weights of best model

    epochs.append(e)
    print("Epoch:", e, "Loss:", loss.item(), "Train Accuracy:", train_accuracy[-1], "Validation Accuracy:", val_accuracy[-1])
    e += 1 #increment epoch counter

  #plotting
  plt.title("Training Loss Curve")
  plt.plot(iterations, losses, label="Train")
  plt.xlabel("iterations")
  plt.ylabel("Loss")
  plt.show()

  plt.title("Training Accuracy Curves")
  plt.plot(epochs, train_accuracy, label="Train")
  plt.plot(epochs, val_accuracy, label="Validation")
  plt.xlabel("iterations")
  plt.ylabel("Training Accuracy")
  plt.legend(loc='best')
  plt.show()

  print("Best validation accuracy:", max_val_acc)
  print("Final training accuracy:", train_accuracy[-1])
  print("Final validation accuracy:", val_accuracy[-1])

In [44]:

train_loader, val_loader, test_loader, classes = load_data(32)

net = BaselineModel()
train_baseline(net, 0.001, 5, train_loader, val_loader)

Done Loading Data


KeyboardInterrupt: 